Script for parsing ESF (Actual)

--------------------------------------------------------------------------------------


Connect to DB


Function read and processing data

In [1]:
import urllib
import os
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Table, MetaData, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session

# Декларативный базовый класс
Base = declarative_base()


# Определение модели для таблицы TRU_Reference
class TRUReference(Base):
    __tablename__ = 'TRU_Reference'

    uid = Column(Integer, primary_key=True, autoincrement=True)
    tru_name = Column(String(255), unique=True, nullable=False)


# Настройка строки подключения

# Конструирование строки подключения
connection_string = ('DRIVER={ODBC Driver 17 for SQL Server};'
                     'SERVER=10.200.100.24;'
                     'DATABASE=Test;'
                     'UID=CRM;'
                     'PWD=2983523456Dare;'
                     'TrustServerCertificate=yes;')

encoded_connection_string = urllib.parse.quote_plus(connection_string)
DATABASE_URL = f'mssql+pyodbc:///?odbc_connect={encoded_connection_string}'
# Создание движка и сессии
engine = create_engine(DATABASE_URL)
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)


# Инициализация базы данных
def init_db(engine):
    Base.metadata.create_all(engine)


init_db(engine)


# Чтение и объединение данных из файлов (Mock функция, замените на свою логику)
def read_and_rename_columns_from_directory(directory_path, column_mapping, dtype_mapping):
    all_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if
                 os.path.isfile(os.path.join(directory_path, f))]
    all_dataframes = []
    file_info = []

    for file in all_files:
        df = pd.read_csv(file, dtype=dtype_mapping)
        df.rename(columns=column_mapping, inplace=True)
        all_dataframes.append(df)
        file_info.append({'file_name': os.path.basename(file), 'row_count': len(df)})

    all_combined_df = pd.concat(all_dataframes, ignore_index=True)
    file_info_df = pd.DataFrame(file_info)

    return all_combined_df, file_info_df


# Вставка уникальных названий TRU в TRU_Reference
def populate_tru_reference_table(df, session):
    print("Вставка уникальных названий TRU в TRU_Reference...")

    unique_tru_names = df['TRU_Name'].dropna().drop_duplicates()

    for name in unique_tru_names:
        exists = session.query(TRUReference).filter_by(tru_name=name).first()
        if not exists:
            tru_reference = TRUReference(tru_name=name)
            session.add(tru_reference)

    session.commit()


# Функция создания таблиц и вставка данных (используется для ESF_FACT_D и ESF_FACT_C)
def create_table_and_insert_data(df, table_name, engine):
    print(f"Создание таблицы '{table_name}' и вставка данных...")
    df.to_sql(table_name, con=engine, index=False, if_exists='replace')


# Вставка информации о файлах
def insert_file_info(file_info, engine):
    print("Вставка информации о файлах в таблицу...")
    file_info.to_sql('file_info', con=engine, index=False, if_exists='replace')


# Объединение DataFrame с UID из TRU_Reference
def add_uid_to_dataframe(df, session):
    print("Объединение DataFrame с UID из TRU_Reference...")
    tru_reference_dict = {tru.tru_name: tru.uid for tru in session.query(TRUReference).all()}
    df['UID'] = df['TRU_Name'].map(tru_reference_dict)
    return df


# Основной блок кода
directory_path = '/Users/dauren/Desktop/02.09.2024Test/'

# Словарь для переименования столбцов
column_mapping = {
    r'Регистрационный номер ЭСФ \(\d{2}/\d{2}/\d{4}\)': 'ESF_Registration_Number',
    'Код органа государственных доходов': 'Tax_Authority_Code',
    'Вид счет-фактуры': 'Invoice_Type',
    'Статус ЭСФ': 'ESF_Status',
    'Статус ЭСФ после отправки получателю': 'ESF_Status_After_Sending',
    'Дата выписки': 'Issue_Date',
    'Дата выписки на бумажном носителе': 'Paper_Issue_Date',
    'Дата совершения оборота': 'Transaction_Date',
    'Ф.И.О., лица выписавшего ЭСФ': 'Issuer_Full_Name',
    'Номер учетной системы': 'System_Number',
    'Дата обновления СФ': 'Invoice_Update_Date',
    'ИИН/БИН': 'IIN_BIN',
    'Наименование поставщика по данным ИНИС на рус.': 'Supplier_Name_INIS',
    'Наименование поставщика по данным ИС ЭСФ': 'Supplier_Name_ESF',
    'Адрес места нахождения': 'Location_Address',
    'Номер свидетельства плательщика НДС': 'VAT_Certificate_Number',
    'Серия свидетельства плательщика НДС': 'VAT_Certificate_Series',
    'Номер документа, подтверждающего поставку ТРУ': 'Supply_Confirm_Document_Number',
    'Дата документа, подтверждающего поставку ТРУ': 'Supply_Confirm_Document_Date',
    'Дополнительные сведения': 'Additional_Information',
    'КБЕ': 'KBE',
    'ИИК': 'IBAN',
    'БИК': 'BIC',
    'Наименование банка': 'Bank_Name',
    'Доля участия': 'Participation_Share',
    'Дата выписки (поставщики)': 'Supplier_Issue_Date',
    'Категория поставщика': 'Supplier_Category',
    'Наименование получателя по данным ИС ЭСФ': 'Receiver_Name_ESF',
    'Наименование получателя по данным ИНИС на рус.': 'Receiver_Name_INIS',
    'ИИН/БИН.1': 'IIN_BIN_1',
    'Адрес места нахождения.1': 'Location_Address_1',
    'Дополнительные сведения.1': 'Additional_Information_1',
    'Код страны': 'Country_Code',
    'Доля участия.1': 'Participation_Share_1',
    'Дата выписки (получатели)': 'Receiver_Issue_Date',
    'Категория получателя': 'Receiver_Category',
    'ИИН/БИН.2': 'IIN_BIN_2',
    'Наименование': 'Name',
    'Адрес отправки': 'Shipping_Address',
    'ИИН/БИН.3': 'IIN_BIN_3',
    'Наименование грузополучателя': 'Consignee_Name',
    'Адрес доставки': 'Delivery_Address',
    'Код страны.1': 'Country_Code_1',
    'Номер договора (контракта) на поставку ТРУ': 'Supply_Contract_Number',
    'Дата договора (контракта) на поставку ТРУ': 'Supply_Contract_Date',
    'Условия оплаты по договору': 'Payment_Terms',
    'Способ отправления': 'Shipping_Method',
    'Номер доверенности на поставку ТРУ': 'Supply_Power_of_Attorney_Number',
    'Дата доверенности на поставку ТРУ': 'Supply_Power_of_Attorney_Date',
    'Пункт назначения': 'Destination_Point',
    'Наличие договора': 'Contract_Availability',
    'ИИК.1': 'IBAN_1',
    'Код ТРУ': 'TRU_Code',
    'Назначение платежа': 'Payment_Purpose',
    'БИК.1': 'BIC_1',
    'Порядковый номер ТРУ': 'TRU_Sequence_Number',
    'Признак происхождения товара, работ, услуг': 'Goods_Origin',
    'Наименование ТРУ': 'TRU_Name',
    'Наименование товаров в соответствии с Декларацией на товары или заявления о ввозе товаров и уплате косвенных налогов': 'Goods_Name_Declaration',
    'Код товара (ТН ВЭД)': 'Commodity_Code',
    'Единица измерения': 'Unit_of_Measurement',
    'Кол-во (объем)': 'Quantity',
    'Цена (тариф) за единицу ТРУ без косвенных налогов': 'Price_per_Unit_Excluding_Taxes',
    'Стоимость товаров, работ, услуг без учета косвенных налогов': 'Cost_Excluding_Taxes',
    'Ставка акциза': 'Excise_Rate',
    'Сумма акциза': 'Excise_Amount',
    'Размер оборота по реализации': 'Turnover_Size',
    'Ставка НДС': 'VAT_Rate',
    'Сумма НДС': 'VAT_Amount',
    'Стоимость товаров, работ, услуг с учетом косвенных налогов': 'Cost_Including_Taxes',
    'Номер декларации на товары, заявления в рамках ТС, СТ-1 или СТ-KZ': 'Declaration_Number',
    'Номер товарной позиции из заявления о ввозе товаров и уплате косвенных налогов или Декларации на товары': 'Item_Position_Number',
    'Дополнительные данные': 'Additional_Data',
    'Курс валюты': 'Currency_Rate',
    'Код валюты': 'Currency_Code',
    'Наименование валюты на рус.': 'Currency_Name',
    'Всего по счету, стоимость товаров, работ, услуг без учета косвенных налогов': 'Total_Cost_Excluding_Taxes',
    'Всего по счету, размер оборота по реализации': 'Total_Turnover_Size',
    'Всего по счету, сумма НДС': 'Total_VAT_Amount',
    'Всего по счету, стоимость товаров, работ, услуг с учетом косвенных налогов': 'Total_Cost_Including_Taxes',
    'ИИН/БИН участника совместной деятельности': 'Joint_Activity_Participant_IIN_BIN',
    'Номер продукта (товара, услуги)': 'Product_Number',
    'Количество (объём)': 'Quantity_Volume',
    'Сумма НДС.1': 'VAT_Amount_1',
    'Стоимость ТРУ с учетом НДС': 'Cost_Including_VAT',
    'Стоимость ТРУ без учета НДС': 'Cost_Excluding_VAT',
    'Размер оборота по реализации.1': 'Turnover_Size_1',
    'Регистрационный номер': 'Registration_Number',
    'Дата выписки СФ, к которому выписывается исправленный/дополнительный СФ': 'Corrected_Invoice_Date'
}

# Словарь для сопоставления названий столбцов с их типами
# Словарь для сопоставления названий столбцов с их типами
dtype_mapping = {
    'ESF_Registration_Number': 'str',
    'Tax_Authority_Code': 'str',
    'Invoice_Type': 'str',
    'ESF_Status': 'str',
    'ESF_Status_After_Sending': 'str',
    'Issue_Date': 'datetime64[ns]',
    'Paper_Issue_Date': 'datetime64[ns]',
    'Transaction_Date': 'datetime64[ns]',
    'Issuer_Full_Name': 'str',
    'System_Number': 'str',
    'Invoice_Update_Date': 'datetime64[ns]',
    'IIN_BIN': 'str',
    'Supplier_Name_INIS': 'str',
    'Supplier_Name_ESF': 'str',
    'Location_Address': 'str',
    'VAT_Certificate_Number': 'str',
    'VAT_Certificate_Series': 'str',
    'Supply_Confirm_Document_Number': 'str',
    'Supply_Confirm_Document_Date': 'datetime64[ns]',
    'Additional_Information': 'str',
    'KBE': 'str',
    'IBAN': 'str',
    'BIC': 'str',
    'Bank_Name': 'str',
    'Participation_Share': 'float',
    'Supplier_Issue_Date': 'datetime64[ns]',
    'Supplier_Category': 'str',
    'Receiver_Name_ESF': 'str',
    'Receiver_Name_INIS': 'str',
    'IIN_BIN_1': 'str',
    'Location_Address_1': 'str',
    'Additional_Information_1': 'str',
    'Country_Code': 'str',
    'Participation_Share_1': 'float',
    'Receiver_Issue_Date': 'datetime64[ns]',
    'Receiver_Category': 'str',
    'IIN_BIN_2': 'str',
    'Name': 'str',
    'Shipping_Address': 'str',
    'IIN_BIN_3': 'str',
    'Consignee_Name': 'str',
    'Delivery_Address': 'str',
    'Country_Code_1': 'str',
    'Supply_Contract_Number': 'str',
    'Supply_Contract_Date': 'datetime64[ns]',
    'Payment_Terms': 'str',
    'Shipping_Method': 'str',
    'Supply_Power_of_Attorney_Number': 'str',
    'Supply_Power_of_Attorney_Date': 'datetime64[ns]',
    'Destination_Point': 'str',
    'Contract_Availability': 'str',
    'IBAN_1': 'str',
    'TRU_Code': 'str',
    'Payment_Purpose': 'str',
    'BIC_1': 'str',
    'TRU_Sequence_Number': 'int',
    'Goods_Origin': 'str',
    'TRU_Name': 'str',
    'Goods_Name_Declaration': 'str',
    'Commodity_Code': 'str',
    'Unit_of_Measurement': 'str',
    'Quantity': 'float',
    'Price_per_Unit_Excluding_Taxes': 'float',
    'Cost_Excluding_Taxes': 'float',
    'Excise_Rate': 'float',
    'Excise_Amount': 'float',
    'Turnover_Size': 'float',
    'VAT_Rate': 'float',
    'VAT_Amount': 'float',
    'Cost_Including_Taxes': 'float',
    'Declaration_Number': 'str',
    'Item_Position_Number': 'str',
    'Additional_Data': 'str',
    'Currency_Rate': 'float',
    'Currency_Code': 'str',
    'Currency_Name': 'str',
    'Total_Cost_Excluding_Taxes': 'float',
    'Total_Turnover_Size': 'float',
    'Total_VAT_Amount': 'float',
    'Total_Cost_Including_Taxes': 'float',
    'Joint_Activity_Participant_IIN_BIN': 'str',
    'Product_Number': 'str',
    'Quantity_Volume': 'float',
    'VAT_Amount_1': 'float',
    'Cost_Including_VAT': 'float',
    'Cost_Excluding_VAT': 'float',
    'Turnover_Size_1': 'float',
    'Registration_Number': 'str',
    'Corrected_Invoice_Date': 'datetime64[ns]'
}

all_df, file_info = read_and_rename_columns_from_directory(directory_path, column_mapping, dtype_mapping)

# Выводим информацию о DataFrame для отладки
print("Количество строк и колонок после чтения и объединения:", all_df.shape)
print("Колонки в объединенном DataFrame:", all_df.columns)
print("Примеры данных:", all_df.head())

if 'TRU_Name' not in all_df.columns:
    raise KeyError("Колонка 'TRU_Name' отсутствует в объединённом DataFrame.")

# Начало сессии
session = Session()

# Вставляем уникальные названия TRU в TRU_Reference
populate_tru_reference_table(all_df, session)

# Создаем таблицу и вставляем данные из объединенного DataFrame в таблицу ESF_FACT_D
create_table_and_insert_data(all_df, 'ESF_FACT_D', engine)

# Создаем таблицу для информации о файлах и вставляем данные
insert_file_info(file_info, engine)

# Объединяем DataFrame с UID из TRU_Reference
all_df_with_uid = add_uid_to_dataframe(all_df, session)

# Убираем ненужный столбец TRU_Name, оставляя только UID
all_df_with_uid.drop(columns=['TRU_Name'], inplace=True)

# Создаем таблицу и вставляем обновленные данные из объединенного DataFrame в таблицу ESF_FACT_C
create_table_and_insert_data(all_df_with_uid, 'ESF_FACT_C', engine)

# Закрытие сессии
session.remove()



/var/folders/q_/zdwkp0rx2qgg_y1xmssy7m100000gn/T/ipykernel_1675/2910293433.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte

Function for create table and insert to data 

Main flow of execution